## Step 1 Preprocess

### Original Data
**AST** : ./csn_dataset/original/csn.pkl <br>
**Code** : ./csn_dataset/orginal/code_tokens_javalang.pkl <br>
**Summary** : ./csn_dataset/original/docstring_tokens_split.pkl <br>

### Transfer pickle file to python2

In [2]:
original_data = ['./csn_dataset/original/csn.pkl',
                './csn_dataset/orginal/code_tokens_javalang.pkl',
                './csn_dataset/original/docstring_tokens_split.pkl']
tranfer_data = ['./csn_dataset/original/drl_csn.pkl',
                './csn_dataset/original/code_tokens_javalang',
                './csn_dataset/original/docstring_tokens.pkl']

# Run here if need tranfer data
# for i in range(3):
#     data = pickle.load(open(original_data[i],'rb'))
#     pickle.dump(data,open(tranfer_data[i], 'rb'), protocol=2)


### Preprocess 
It cost about 12 hour and default set <br>
code length is 100, <br>
summary length is 10, <br>
vocab size is 50000 <br>
**log is ./csn_dataset/train_data0330/log.preprocess'**
``` shell
python run.py preprocess
```


In [2]:
import torch
# output is a file with fid
data_with_fid_path = './csn_dataset/train_data/processed.train.pt'
data_with_fid =  torch.load(data_with_fid_path)

In [3]:
data_with_fid.keys()

dict_keys(['train_xe', 'test', 'train_pg', 'valid', 'dicts'])

In [12]:
print(data_with_fid['train_xe'][0].keys())


dict_keys(['src', 'tgt', 'trees'])


In [13]:
print(len(data_with_fid['train_xe']))
print(len(data_with_fid['test']))
print(len(data_with_fid['valid']))

430585
25432
14436


### Filter leaf_count = 1
filter_data is remove data which leaf_count = 1 and remove fid info ( it fit original train code) <br>
verify data is verify filter data is right or not

```shell
python filter_data.py
python verify_data.py
```

In [8]:
# filter data output 
filter_data_path = '/mnt/yuxuan/Code_summarization0330/csn_dataset/train_data0330/processed_filter.train.pt2'
filter_data = torch.load(filter_data_path)

In [9]:
filter_data.keys()

dict_keys(['train_xe', 'test', 'train_pg', 'valid', 'dicts'])

In [11]:
filter_data['train_xe'].keys()

dict_keys(['src', 'trees', 'tgt'])

In [14]:
print(len(filter_data['train_xe']['src']))
print(len(filter_data['test']['src']))
print(len(filter_data['valid']['src']))

430581
25431
14423


## Step 2 Train model

### parameter set
**different with original** <br>
rnn_size : 512 -> 256 <br>
word_vec_size : 512 -> 256 <br>
batch_size : 64 -> 256 <br>

**Other parameter set** <br>
a2c-train.py get_opt()

### Train
```
python run.py train_a2c 4 10 3 hybrid 1 0
```

### Explore tree length

#### build_test_data
output is ./csn_dataset/test_diff_tree_depth
```
python build_test_tree_depth.py
```

#### train test
output is ./csn_dataset/test_diff_tree_depth_out
```
python run.py test_tree_depth 3 5 1 hybrid 1 0
```

### Other detail
Other datail is shown on explore\ tree\ depth.ipynb

## 3. Result

### log dir
log is saved on 

./csn_dataset/train_data_out/log.a2c-train_%s_%s_%s_%s_%s_g%s_model <br>
The mean of about %s is: <br>
start_reinforce, end_epoch, critic_pretrain_epochs, data_type, has_attn, gpus

### diff epoch set
There have three part of model<br>
1. actor
2. critic
3. reinforce

CU02: 3 3 4 <br>
MSRA57: 6 6 6 <br>
Azure157: 10 10 10 <br>